# 微分可能LUTモデルによるMNIST学習

Stochasticモデルに BatchNormalization や Binarize(backward時はHard-Tanh)を加えることで、より一般的なデータに対してLUT回路学習を行います。 

## 事前準備

In [28]:
import os
import shutil
import numpy as np
from tqdm.notebook import tqdm

import torch
import torchvision
import torchvision.transforms as transforms

import binarybrain as bb

異なる閾値で2値化した画像でフレーム数を水増ししながら学習させます。この水増しをバイナリ変調と呼んでいます。

ここではフレーム方向の水増し量を frame_modulation_size で指定しています。

In [29]:
# configuration
data_path             = './data/'
net_name              = 'MnistDifferentiableLut4Simple'
data_path             = os.path.join('./data/', net_name)
rtl_sim_path          = '../../verilog/mnist/tb_mnist_lut4_simple'
rtl_module_name       = 'MnistLutSimple'
output_velilog_file   = os.path.join(data_path, net_name + '.v')
sim_velilog_file      = os.path.join(rtl_sim_path, rtl_module_name + '.v')

epochs                = 4
mini_batch_size       = 64*2
frame_modulation_size = 31

データセットは PyTorch の torchvision を使います。ミニバッチのサイズも DataLoader で指定しています。
BinaryBrainではミニバッチをフレーム数として FrameBufferオブジェクトで扱います。
バイナリ変調で計算中にフレーム数が変わるためデータセットの準備観点でのミニバッチと呼び分けています。

In [30]:
# dataset
dataset_path = './data/'
dataset_train = torchvision.datasets.MNIST(root=dataset_path, train=True, transform=transforms.ToTensor(), download=True)
dataset_test  = torchvision.datasets.MNIST(root=dataset_path, train=False, transform=transforms.ToTensor(), download=True)
loader_train = torch.utils.data.DataLoader(dataset=dataset_train, batch_size=mini_batch_size, shuffle=True, num_workers=2)
loader_test  = torch.utils.data.DataLoader(dataset=dataset_test,  batch_size=mini_batch_size, shuffle=False, num_workers=2)

## ネットワークの構築

DifferentiableLut に特に何もオプションをつけなければOKです。<br>
バイナリ変調を施すためにネットの前後に RealToBinary層とBinaryToReal層を入れています。<br>
send_command で "binary true" を送ることで、DifferentiableLut の内部の重み係数が 0.0-1.0 の間に拘束されます。

接続数がLUTの物理構成に合わせて、1ノード当たり6個なので層間で6倍以上ノード数が違うと接続されないノードが発生するので、注意してネットワーク設計が必要です。
最終段は各クラス7個の結果を出して Reduce で足し合わせています。こうすることで若干の改善がみられるとともに、加算結果が INT3 相当になるために若干尤度を数値的に見ることができるようです。

In [31]:
# define network
net = bb.Sequential([
            bb.RealToBinary(frame_modulation_size=frame_modulation_size),
            bb.DifferentiableLut([64*4*4], N=4),
            bb.DifferentiableLut([64*4], N=4, connection='serial'),
            bb.DifferentiableLut([64], N=4, connection='serial'),
            bb.DifferentiableLut([10*4*4], N=4),
            bb.DifferentiableLut([10*4], N=4, connection='serial'),
            bb.DifferentiableLut([10], N=4, connection='serial'),
            bb.BinaryToReal(frame_integration_size=frame_modulation_size)
        ])
net.set_input_shape([1, 28, 28])

net.send_command("binary true")

loss      = bb.LossSoftmaxCrossEntropy()
metrics   = bb.MetricsCategoricalAccuracy()
optimizer = bb.OptimizerAdam()

optimizer.set_variables(net.get_parameters(), net.get_gradients())

## 学習の実施

load_networks/save_networks で途中結果を保存/復帰可能できます。ネットワークの構造が変わると正常に読み込めなくなるので注意ください。
(その場合は新しいネットをsave_networksするまで一度load_networks をコメントアウトください)

tqdm などを使うと学習過程のプログレス表示ができて便利です。

In [32]:
bb.load_networks(data_path, net)

print(net.get_info())

# learning
for epoch in range(epochs):
    # learning
    loss.clear()
    metrics.clear()
    with tqdm(loader_train) as t:
        for images, labels in t:
            x_buf = bb.FrameBuffer.from_numpy(np.array(images).astype(np.float32))
            t_buf = bb.FrameBuffer.from_numpy(np.identity(10)[np.array(labels)].astype(np.float32))

            y_buf = net.forward(x_buf, train=True)
            
            dy_buf = loss.calculate(y_buf, t_buf)
            metrics.calculate(y_buf, t_buf)
            
            net.backward(dy_buf)

            optimizer.update()
            
            t.set_postfix(loss=loss.get(), acc=metrics.get())

    # test
    loss.clear()
    metrics.clear()
    for images, labels in loader_test:
        x_buf = bb.FrameBuffer.from_numpy(np.array(images).astype(np.float32))
        t_buf = bb.FrameBuffer.from_numpy(np.identity(10)[np.array(labels)].astype(np.float32))

        y_buf = net.forward(x_buf, train=False)

        loss.calculate(y_buf, t_buf)
        metrics.calculate(y_buf, t_buf)

    print('epoch[%d] : loss=%f accuracy=%f' % (epoch, loss.get(), metrics.get()))

    bb.save_networks(data_path, net)

----------------------------------------------------------------------
[Sequential] 
 input  shape : [1, 28, 28] output shape : [10]
  --------------------------------------------------------------------
  [RealToBinary] 
   input  shape : {1, 28, 28} output shape : {1, 28, 28}
  --------------------------------------------------------------------
  [DifferentiableLut4] 
   input  shape : {1, 28, 28} output shape : {1024}
   binary : 1   batch_norm : 1
  --------------------------------------------------------------------
  [DifferentiableLut4] 
   input  shape : {1024} output shape : {256}
   binary : 1   batch_norm : 1
  --------------------------------------------------------------------
  [DifferentiableLut4] 
   input  shape : {256} output shape : {64}
   binary : 1   batch_norm : 1
  --------------------------------------------------------------------
  [DifferentiableLut4] 
   input  shape : {64} output shape : {160}
   binary : 1   batch_norm : 1
  -----------------------------

  0%|          | 0/469 [00:00<?, ?it/s]

epoch[0] : loss=1.696531 accuracy=0.784500


  0%|          | 0/469 [00:00<?, ?it/s]

epoch[1] : loss=1.670991 accuracy=0.819700


  0%|          | 0/469 [00:00<?, ?it/s]

epoch[2] : loss=1.725729 accuracy=0.746700


  0%|          | 0/469 [00:00<?, ?it/s]

epoch[3] : loss=1.697923 accuracy=0.793800


## FPGA用Verilog出力

最後に学習したネットワークを Verilog 出力します。
MNISTのサイズである 28x28=784bit の入力を 10bit の分類をして出力するだけのシンプルなモジュールを出力します。

In [33]:
# export verilog
bb.load_networks(data_path, net)

os.makedirs(data_path, exist_ok=True)
os.makedirs(rtl_sim_path, exist_ok=True)

# 結果を出力
with open(output_velilog_file, 'w') as f:
    f.write('`timescale 1ns / 1ps\n\n')
    bb.dump_verilog_lut_layers(f, module_name=rtl_module_name, net=net)

# Simulation用ファイルに上書きコピー
shutil.copyfile(output_velilog_file, sim_velilog_file)

'../../verilog/mnist/tb_mnist_lut4_simple/MnistLutSimple.v'

In [34]:
# シミュレーション用データファイル作成
with open(os.path.join(rtl_sim_path, 'mnist_test.txt'), 'w') as f:
    bb.dump_verilog_readmemb_image_classification(f ,loader_test)

## モデルの内部の値を取得する

Verilog以外の言語やFPGA以外に適用したい場合、接続とLUTテーブルの2つが取得できれば同じ計算をするモデルをインプリメントすることが可能です。

### 事前準備
そのままだと勾配はリセットされているので少しだけ逆伝搬を実施します

In [35]:
# 最新の保存データ読み込み
bb.load_networks(data_path, net)

# layer を取り出す
layer0 = net[1]
layer1 = net[2]
layer2 = net[3]

### 接続を取得する

LUTモデルは get_connection_list() にて接続行列を取得できます。<br>
ここでの各出力ノードは、6つの入力と接続されており、layer0 の出力ノードは 1024 個あるので、1024x4 の行列が取得できます。

In [36]:
connection_mat = np.array(layer0.get_connection_list())
print(connection_mat.shape)
connection_mat

(1024, 4)


array([[531, 302, 316, 420],
       [736, 111,  73, 191],
       [529, 203, 400,  20],
       ...,
       [ 96, 287, 745, 140],
       [ 39,  75, 762,  24],
       [730, 179,  33, 337]])

### FPGA化する場合のLUTテーブルを取得する

LUT化する場合のテーブルを取得します。<br>
6入力のLUTモデルなので $ 2^4 = 16 $ 個のテーブルがあります。<br>
モデル内に BatchNormalization 等を含む場合はそれらも加味して最終的にバイナリLUTにする場合に適した値を出力します。

In [37]:
lut_mat = np.array(layer0.get_lut_table_list())
print(lut_mat.shape)
lut_mat

(1024, 16)


array([[False,  True,  True, ..., False,  True,  True],
       [False, False, False, ...,  True,  True,  True],
       [ True, False, False, ..., False, False, False],
       ...,
       [False,  True,  True, ...,  True, False,  True],
       [False,  True,  True, ...,  True,  True,  True],
       [False, False,  True, ..., False, False, False]])

### 重み行列を覗いてみる

6入力のLUTモデルなので $ 2^4 = 16 $ 個のテーブルがあります。<br>
W() にて bb.Tensor 型で取得可能で、numpy() にて ndarray に変換できます。

In [44]:
W = layer0.W().numpy()
print(W.shape)
W

(1024, 16)


array([[0.49162844, 0.5280779 , 0.57371926, ..., 0.34074423, 0.6116431 ,
        0.607071  ],
       [0.4783129 , 0.19335054, 0.47556198, ..., 0.5076589 , 0.5880844 ,
        0.5181765 ],
       [0.61731434, 0.4684478 , 0.3765769 , ..., 0.49074215, 0.41823113,
        0.3487906 ],
       ...,
       [0.45175382, 0.6535614 , 0.53194654, ..., 0.6134853 , 0.40244558,
        0.52635974],
       [0.36481518, 0.50609213, 0.5197173 , ..., 0.54208535, 0.5630838 ,
        0.59852237],
       [0.49354315, 0.499773  , 0.5950174 , ..., 0.43302473, 0.5230216 ,
        0.52889705]], dtype=float32)

### 勾配を覗いてみる

同様に dW() でW の勾配が取得できます

In [39]:
# そのままだとすべて0なので、1回だけbackward実施
for images, labels in loader_test:
    x_buf = bb.FrameBuffer.from_numpy(np.array(images).astype(np.float32))
    t_buf = bb.FrameBuffer.from_numpy(np.identity(10)[np.array(labels)].astype(np.float32))
    y_buf = net.forward(x_buf, train=True)
    net.backward(loss.calculate(y_buf, t_buf))
    break

dW = layer0.dW().numpy()
print(dW.shape)
dW

(1024, 16)


array([[ 7.5323507e-05,  2.5107329e-05, -1.3911631e-07, ...,
        -3.6132101e-06, -1.4222352e-05, -4.7407748e-06],
       [-5.4312404e-06,  2.4563633e-07, -1.8090359e-06, ...,
         6.0351158e-07,  3.7524296e-07,  2.0110929e-07],
       [-4.3391756e-04, -1.4515530e-04, -2.1268107e-04, ...,
         5.6851361e-05,  4.5495901e-05,  1.3617726e-05],
       ...,
       [ 2.1891750e-04,  1.4854758e-04, -2.4731923e-04, ...,
         1.1613833e-05, -3.3771503e-05, -8.4581043e-06],
       [ 1.1175871e-08,  8.3819032e-09,  8.3819032e-09, ...,
         6.9849193e-10,  6.9849193e-10,  2.0372681e-10],
       [ 3.3061951e-08,  1.1175871e-08, -3.0093361e-08, ...,
         1.1568773e-09, -3.2450771e-09, -1.0877557e-09]], dtype=float32)

## FPGA用HLS(C言語高位合成)で使う為の出力

内部データを取得する例としてHSL(C言語高位合成)用の出力を作ってみます

In [40]:
def make_lut_func_name(name, node):
    return "%s_lut_%d"%(name, node)

def dump_hls_lut_node2(f, name, lut, node):
    f.write("\ninline ap_uint<1> %s(\n"%(make_lut_func_name(name, node)))
    n = lut.get_node_connection_size(node)
    s = lut.get_lut_table_size(node)
    for i in range(n):
        f.write("        ap_uint<1> in_data%d"%(i))
        if i < n-1:
            f.write(",\n")
        else:
            f.write(")\n")
    f.write("{\n")
    f.write("#pragma HLS inline\n\n")
    f.write("    ap_uint<%d> index;\n"%(n))
    for i in range(n):
        f.write("    index[%d] = in_data%d;\n"%(i, i))
    f.write("    \n")
    f.write("    ap_uint<1> table[%d] = {"%(s))
    for i in range(s):
        f.write("%d,"%(lut.get_lut_table(node ,i)))
    f.write("};\n")
#    for i in range(s):
#        f.write("    table[%d] = %d;\n"%(i, lut.get_lut_table(node ,i)))
#    f.write("    \n")
#   f.write("    #pragma HLS resource variable=table core=ROM_1P_LUTRAM\n")
    f.write("    #pragma HLS bind_storage variable=table type=ROM_1P impl=LUTRAM\n")
    f.write("    return table[index];\n")
    f.write("}\n\n")

def dump_hls_lut_node(f, name, lut, node):
    f.write("\ninline ap_uint<1> %s(\n"%(make_lut_func_name(name, node)))
    n = lut.get_node_connection_size(node)
    s = lut.get_lut_table_size(node)
    
    tbl = 0
    for i in range(s):
        if lut.get_lut_table(node ,i):
            tbl += (1 << i)
    
    for i in range(n):
        f.write("        ap_uint<1> in_data%d"%(i))
        if i < n-1:
            f.write(",\n")
        else:
            f.write(")\n")
    f.write("{\n")
    f.write("#pragma HLS inline\n")
    f.write("    ap_uint<%d> index;\n"%(n))
    for i in range(n):
        f.write("    index[%d] = in_data%d;\n"%(i, i))
    f.write("    ap_uint<%d> table= 0x%016xLL;\n"%(s, tbl))
    f.write("    return table[index];\n")
    f.write("}\n\n")

def dump_hls_lut(f, name, lut):
    ins  = lut.get_input_node_size()
    outs = lut.get_output_node_size()
    for node in range(outs):
        dump_hls_lut_node2(f, name, lut, node)
    
    f.write("\n")
    f.write("inline ap_uint<%d> %s(ap_uint<%d> in_data)\n"%(outs, name, ins))
    f.write("{\n")
    f.write("    ap_uint<%d>  out_data;\n"%(outs))
    for node in range(outs):
        f.write("    out_data[%d] = %s("%(node, make_lut_func_name(name, node)))
        n = lut.get_node_connection_size(node)
        for i in range(n):
            f.write("in_data[%d]"%(lut.get_node_connection_index(node, i)))
            if i < n-1: 
                f.write(",")
            else:
                f.write(");\n")
    f.write("    return out_data;\n")   
    f.write("}\n\n")

# 学習済みを読みなおす    
bb.load_networks(data_path, net)
with open("MnistDifferentiableLutSimpleHls.h", "w") as f:
    f.write('#include "ap_int.h"\n\n')
    for i in range(1, 7):
        dump_hls_lut(f, "mnist_layer%d"%i, net[i])

In [41]:
num = 0
with open('mnist_hls_test.txt', 'w') as f:
    for images, labels in loader_test:
        x_buf = np.array(images).astype(np.float32)
        t_buf = np.array(labels)
        for i in range(x_buf.shape[0]):
            f.write("%d"%t_buf[i])
            for y in range(x_buf.shape[2]):
                for x in range(x_buf.shape[3]):
                    f.write(" %d"%(x_buf[i, 0, y, x] > 0.5))
            f.write("\n")
            num += 1
        if num > 1024:
            break

In [42]:
! copy MnistDifferentiableLutSimpleHls.h \\wsl$\Ubuntu-20.04\home\ryuji\git-work\jelly_develop\projects\ultra96v2_hls_test\hls_\src
! copy mnist_hls_test.txt \\wsl$\Ubuntu-20.04\home\ryuji\git-work\jelly_develop\projects\ultra96v2_hls_test\hls_\testbench


/bin/bash: copy: command not found
/bin/bash: copy: command not found


In [43]:
! copy MnistDifferentiableLutSimpleHls.h ..\..\hls\mnist\mnist_simple\src
! copy mnist_hls_test.txt                ..\..\hls\mnist\mnist_simple\testbench

/bin/bash: copy: command not found
/bin/bash: copy: command not found
